In [1]:
import numpy as np
import pandas as pd

import re # regular expression
import os # operating system
import shutil # shell utilities
import fnmatch # file name matching

Check the occurrence of Tags

In [72]:
maindir='../GCv11-02/Code_FlexGrid_v11-02'

Tags = ['GEOS_FP','MERRA2','FLEXGRID']

# match something like #if defined( GEOS_FP ) or #elif defined ( MERRA2)
# white spaces can occur at many places! Use ( )* to represent them
regex_list = [re.compile(r'#(.*)' +
                         r'defined( )*'+
                         r'\(( )*' +
                         Tag + 
                         r'( )*\)' ) 
              for Tag in Tags]  
n_tag = len(Tags)

fileFormat = '*.F*' # .F or .F90

data = [] # a list of list, to be converted to pandas DataFrame
for dirpath, dirs, files in os.walk(maindir):

    for filename in fnmatch.filter(files, fileFormat):
    
        # contruct the full path
        filepath = os.path.join(dirpath, filename)
        
        # record current file, followed by n_tag 0s for later use
        data.append( [dirpath[len(maindir):],filename] + [0]*n_tag )
        
        # read the file line by line
        with open(filepath,"r",encoding = "ISO-8859-1") as file:
            for line in file:
                for n,regex in enumerate(regex_list):          
                    if regex.match(line):
                        data[-1][n+2] += 1
        
df_raw = pd.DataFrame(data, columns=['dir','filename']+Tags)

df = df_raw[(df_raw['GEOS_FP'] > 0) | (df_raw['MERRA2'] > 0)]
df.reset_index(drop=True,inplace=True)

#df[df["GEOS_FP"] != df["MERRA2"]]
df

,dir,filename,GEOS_FP,MERRA2,FLEXGRID
0,/GeosUtil,pressure_mod.F,1,1,1
1,/GeosUtil,transfer_mod.F,6,6,5
2,/GeosUtil,time_mod.F,1,1,1
3,/GeosUtil,bpch2_mod.F,3,4,0
4,/Headers,state_met_mod.F90,4,4,4
5,/Headers,CMN_SIZE_mod.F,5,5,0
6,/GeosCore,carbon_mod.F,3,3,3
7,/GeosCore,tpcore_window_mod.F90,1,1,0
8,/GeosCore,tpcore_bc_mod.F,4,4,0
9,/GeosCore,gamap_mod.F,1,1,1


Count lines like

defined( GEOS_FP ) || defined( MERRA2 ) 

or #if !defined( MERRA ) && !defined( GEOS_FP )

In [22]:
# mostly copy from the previous cell

maindir='../GCv11-02/Code_FlexGrid_v11-02'

Tags = ['GEOS_FP','MERRA2']

# match something like #if defined( GEOS_FP ) or #elif defined ( MERRA2)
# white spaces can occur at many places! Use ( )* to represent them
regex_list = [re.compile(r'#(.*)' +
                         r'defined( )*'+
                         r'\(( )*' +
                         Tag + 
                         r'( )*\)' ) 
              for Tag in Tags]  

fileFormat = '*.F*' # .F or .F90

data = [] # a list of list, to be converted to pandas DataFrame
for dirpath, dirs, files in os.walk(maindir):

    for filename in fnmatch.filter(files, fileFormat):
    
        # contruct the full path
        filepath = os.path.join(dirpath, filename)
        
        # record current file, followed by n_tag 0s for later use
        data.append( [dirpath[len(maindir):],filename] + [0]*2)
        
        # read the file line by line
        with open(filepath,"r",encoding = "ISO-8859-1") as file:
            for line in file:
                if regex_list[0].match(line) and regex_list[1].match(line):
                    if "||" in line:
                        data[-1][2] += 1
                        print(line)
                    if "&&" in line:
                        data[-1][3] += 1
                        print(line)
        
df_raw = pd.DataFrame(data, columns=['dir','filename','||','!&&']  )

df = df_raw[(df_raw['||'] > 0) | (df_raw['!&&'] > 0)]

df.reset_index(drop=True,inplace=True)

df

#if   defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 )

#elif defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 )

#elif defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 )

#if   defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 )

#if   defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 )

#if   defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 )

#if   defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 )

#elif defined( GEOS_FP ) || defined( MERRA ) || defined( MERRA2 )

#elif defined( GEOS_FP ) || defined( MERRA ) || defined( MERRA2 )

#if defined( GEOS_FP ) || defined( MERRA2 )

#elif defined( GEOS_FP ) || defined( MERRA ) || defined( MERRA2 )

#if   defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 )

#if   defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || de

,dir,filename,||,!&&
0,/GeosUtil,pressure_mod.F,1,0
1,/GeosUtil,transfer_mod.F,5,0
2,/GeosUtil,time_mod.F,1,0
3,/Headers,state_met_mod.F90,4,0
4,/GeosCore,carbon_mod.F,2,1
5,/GeosCore,gamap_mod.F,1,0
6,/GeosCore,strat_chem_mod.F90,1,0
7,/GeosCore,convection_mod.F,4,0
8,/GeosCore,tagged_o3_mod.F,1,0
9,/GeosCore,diagnostics_mod.F90,1,0


Automatically change all lines including

defined( GEOS_FP ) || defined( MERRA2 )

to

defined( GEOS_FP ) || defined( MERRA2 ) || defined( FLEXGRID )

This should cover most of files. Manually change other codes.

- https://stackoverflow.com/questions/30319701/add-lines-to-text-file-after-occurence-of-certain-text

In [70]:
# warning: this cell will modify the code files!
# use git reset --hard to recover the orginal state

maindir='../GCv11-02/Code_FlexGrid_v11-02'

Tags = ['GEOS_FP','MERRA2']

# match something like #if defined( GEOS_FP ) or #elif defined ( MERRA2)
# white spaces can occur at many places! Use ( )* to represent them
regex_list = [re.compile(r'#(.*)' +
                         r'defined( )*'+
                         r'\(( )*' +
                         Tag + 
                         r'( )*\)' ) 
              for Tag in Tags]  

fileFormat = '*.F*' # .F or .F90

for dirpath, dirs, files in os.walk(maindir):

    for filename in fnmatch.filter(files, fileFormat):
    
        # contruct the full path
        filepath = os.path.join(dirpath, filename)
        
        match = False # assume this file doest not contain the pattern
        
        # read the file line by line
        with open(filepath,"r",encoding = "ISO-8859-1") as file:
            
            # read all lines, modify lines in-memory, and finally overwrite original file
            # this approach could be relative slow but clean.
            lines = file.readlines()
            
            for n,line in enumerate(lines):
                
                # the last character is always linebreak. Skip it.
                if regex_list[0].match(line) and regex_list[1].match(line):
                    if "||" in line:
                        print(filepath);print(line)
                        templine = line[:-1]+' || defined( FLEXGRID )\n'
                        print(templine)
                        lines[n]=templine
                        match = True
                    if "&&" in line:
                        print(filepath);print(line)
                        templine = line[:-1]+' && ! defined( FLEXGRID )\n'
                        print(templine)
                        lines[n]=templine
                        match = True
                        
        # open the same file to overwrite
        if match:
            with open(filepath,"w",encoding = "ISO-8859-1") as file:
                for line in lines:
                    file.write(line)

../GCv11-02/Code_FlexGrid_v11-02/GeosUtil/pressure_mod.F
#if   defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 )

#if   defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 ) || defined( FLEXGRID )

../GCv11-02/Code_FlexGrid_v11-02/GeosUtil/transfer_mod.F
#elif defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 )

#elif defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 ) || defined( FLEXGRID )

../GCv11-02/Code_FlexGrid_v11-02/GeosUtil/transfer_mod.F
#elif defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 )

#elif defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 ) || defined( FLEXGRID )

../GCv11-02/Code_FlexGrid_v11-02/GeosUtil/transfer_mod.F
#if   defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 )

#if   defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 ) 